In [1]:
from core.load_data import DownloadDadosTse
from core.utils import solve_path
import pandas as pd

In [2]:
tse = DownloadDadosTse()

In [3]:
df_perfil = tse('perfil_sp', encoding='latin-1', sep=';')

In [4]:
df_perfil.head()

,DT_GERACAO,HH_GERACAO,ANO_ELEICAO,SG_UF,CD_MUNICIPIO,NM_MUNICIPIO,CD_MUN_SIT_BIOMETRIA,DS_MUN_SIT_BIOMETRIA,NR_ZONA,NR_SECAO,...,CD_ESTADO_CIVIL,DS_ESTADO_CIVIL,CD_FAIXA_ETARIA,DS_FAIXA_ETARIA,CD_GRAU_ESCOLARIDADE,DS_GRAU_ESCOLARIDADE,QT_ELEITORES_PERFIL,QT_ELEITORES_BIOMETRIA,QT_ELEITORES_DEFICIENCIA,QT_ELEITORES_INC_NM_SOCIAL
0,01/11/2022,03:40:49,9999,SP,61204,ILHA SOLTEIRA,-2,Não se aplica,368,86,...,1,SOLTEIRO,5054,50 a 54 anos,3,ENSINO FUNDAMENTAL INCOMPLETO,1,1,0,0
1,01/11/2022,03:40:49,9999,SP,61204,ILHA SOLTEIRA,-2,Não se aplica,368,86,...,3,CASADO,3034,30 a 34 anos,5,ENSINO MÉDIO INCOMPLETO,2,2,0,0
2,01/11/2022,03:40:49,9999,SP,61204,ILHA SOLTEIRA,-2,Não se aplica,368,86,...,3,CASADO,4549,45 a 49 anos,5,ENSINO MÉDIO INCOMPLETO,2,2,0,0
3,01/11/2022,03:40:49,9999,SP,61204,ILHA SOLTEIRA,-2,Não se aplica,368,86,...,3,CASADO,5054,50 a 54 anos,3,ENSINO FUNDAMENTAL INCOMPLETO,1,1,0,0
4,01/11/2022,03:40:49,9999,SP,61204,ILHA SOLTEIRA,-2,Não se aplica,368,86,...,3,CASADO,7074,70 a 74 anos,3,ENSINO FUNDAMENTAL INCOMPLETO,2,2,0,0


In [5]:
df_perfil = df_perfil[df_perfil['CD_MUNICIPIO']==71072]

In [6]:
df_perfil.keys()

Index(['DT_GERACAO', 'HH_GERACAO', 'ANO_ELEICAO', 'SG_UF', 'CD_MUNICIPIO',
       'NM_MUNICIPIO', 'CD_MUN_SIT_BIOMETRIA', 'DS_MUN_SIT_BIOMETRIA',
       'NR_ZONA', 'NR_SECAO', 'CD_GENERO', 'DS_GENERO', 'CD_ESTADO_CIVIL',
       'DS_ESTADO_CIVIL', 'CD_FAIXA_ETARIA', 'DS_FAIXA_ETARIA',
       'CD_GRAU_ESCOLARIDADE', 'DS_GRAU_ESCOLARIDADE', 'QT_ELEITORES_PERFIL',
       'QT_ELEITORES_BIOMETRIA', 'QT_ELEITORES_DEFICIENCIA',
       'QT_ELEITORES_INC_NM_SOCIAL'],
      dtype='object')

In [7]:
df_perfil['DT_GERACAO'].unique()

array(['01/11/2022'], dtype=object)

In [8]:
df_perfil['ANO_ELEICAO'].unique()

array([9999])

In [9]:
df_perfil['DS_MUN_SIT_BIOMETRIA'].unique()

array(['Não se aplica'], dtype=object)

In [10]:
colunas_interesse = [
    'DS_GENERO',
    'DS_ESTADO_CIVIL',
    'DS_FAIXA_ETARIA',
    'DS_GRAU_ESCOLARIDADE',
]
val_col = 'QT_ELEITORES_PERFIL'
index_col = 'NR_ZONA'

In [11]:
group_keys = [index_col] + colunas_interesse

In [12]:
total_keys = group_keys + [val_col, 'NR_SECAO'] 

In [13]:
df_perfil = df_perfil[total_keys].copy()

In [14]:
df_perfil.duplicated().any()

False

In [15]:
total_zonas = len(df_perfil['NR_ZONA'].unique())
total_zonas

58

In [16]:
grouped_zona = df_perfil.groupby(group_keys).sum()[val_col].reset_index()

In [17]:
grouped_zona.sample(3)

,NR_ZONA,DS_GENERO,DS_ESTADO_CIVIL,DS_FAIXA_ETARIA,DS_GRAU_ESCOLARIDADE,QT_ELEITORES_PERFIL
29846,325,MASCULINO,CASADO,21 a 24 anos,SUPERIOR COMPLETO,3
15415,251,MASCULINO,SOLTEIRO,30 a 34 anos,ENSINO MÉDIO INCOMPLETO,940
54516,392,FEMININO,SEPARADO JUDICIALMENTE,55 a 59 anos,ENSINO MÉDIO INCOMPLETO,8


In [18]:
grouped_zona.duplicated().any()

False

In [19]:
grouped_zona['DS_ESTADO_CIVIL'].unique()

array(['CASADO', 'DIVORCIADO', 'NÃO INFORMADO', 'SEPARADO JUDICIALMENTE',
       'SOLTEIRO', 'VIÚVO'], dtype=object)

In [20]:
dfs = []
for col in colunas_interesse:
    grouped_col = grouped_zona.groupby([index_col, 
                                        col])[val_col].sum().reset_index()
    pivoted_col = pd.pivot_table(grouped_col, 
                                 index='NR_ZONA', 
                                 columns = col, 
                                 values='QT_ELEITORES_PERFIL')
    col_vals = [col for col in pivoted_col.columns if col!='NR_ZONA']
    pivoted_col = pivoted_col.reset_index()
    pivoted_col[f'total_{col.lower()}'] = pivoted_col[col_vals].sum(axis=1).astype(int)
    #checa se tamanho ficou certo
    assert len(pivoted_col)==total_zonas
    
    rename = {col_val:f"{col.lower().replace('ds_', '')}_{col_val.lower()}"
             for col_val in col_vals}
    pivoted_col.rename(rename, axis=1, inplace=True)
    
    dfs.append(pivoted_col)
    
df_final = pd.concat(dfs, axis=1)

In [21]:
cols_totais = [col for col in df_final.columns
              if col.startswith('total_')]

In [22]:
cols_totais

['total_ds_genero',
 'total_ds_estado_civil',
 'total_ds_faixa_etaria',
 'total_ds_grau_escolaridade']

In [23]:
#checar se todas as colunas de total somam a mesma coisa
df_totais = df_final[cols_totais]
assert df_totais.sum().duplicated(keep=False).all()
col_total = df_totais.iloc[:,0]

df_final['total_zona'] = col_total

df_final.drop(cols_totais, axis=1, inplace=True)

In [24]:
#checar se todas as zonas sao iguais
df_zonas = df_final['NR_ZONA']
assert df_zonas.sum().duplicated(keep=False).all()
col_zona = df_zonas.iloc[:,0]

df_final['nr_zona'] = col_zona

df_final.drop('NR_ZONA', axis=1, inplace=True)

In [25]:
del df_totais
del grouped_zona

In [26]:
df_final.keys()

Index(['genero_feminino', 'genero_masculino', 'genero_não informado',
       'estado_civil_casado', 'estado_civil_divorciado',
       'estado_civil_não informado', 'estado_civil_separado judicialmente',
       'estado_civil_solteiro', 'estado_civil_viúvo',
       'faixa_etaria_100 anos ou mais', 'faixa_etaria_16 anos',
       'faixa_etaria_17 anos', 'faixa_etaria_18 anos', 'faixa_etaria_19 anos',
       'faixa_etaria_20 anos', 'faixa_etaria_21 a 24 anos',
       'faixa_etaria_25 a 29 anos', 'faixa_etaria_30 a 34 anos',
       'faixa_etaria_35 a 39 anos', 'faixa_etaria_40 a 44 anos',
       'faixa_etaria_45 a 49 anos', 'faixa_etaria_50 a 54 anos',
       'faixa_etaria_55 a 59 anos', 'faixa_etaria_60 a 64 anos',
       'faixa_etaria_65 a 69 anos', 'faixa_etaria_70 a 74 anos',
       'faixa_etaria_75 a 79 anos', 'faixa_etaria_80 a 84 anos',
       'faixa_etaria_85 a 89 anos', 'faixa_etaria_90 a 94 anos',
       'faixa_etaria_95 a 99 anos', 'faixa_etaria_inválido',
       'grau_escolaridad

Agora vou criar as colunas para a modelagem

In [27]:
df_modelagem = pd.DataFrame()

In [28]:
df_modelagem['nr_zona'] = df_final['nr_zona']

In [29]:
df_modelagem['prop_mulheres'] = df_final['genero_feminino']/df_final['total_zona']

In [30]:
df_modelagem['prop_superior_completo'] = df_final['grau_escolaridade_superior completo']\
                                        /df_final['total_zona']

In [31]:
escol_menos_fund = [
    'grau_escolaridade_analfabeto',
    'grau_escolaridade_lê e escreve',
    'grau_escolaridade_ensino fundamental incompleto',
    ]

In [32]:
df_modelagem['prop_sem_fundametal'] = df_final[escol_menos_fund].sum(axis=1)/df_final['total_zona']

In [33]:
cols_mais_60 = [
    'faixa_etaria_60 a 64 anos',
    'faixa_etaria_65 a 69 anos', 
    'faixa_etaria_70 a 74 anos',
    'faixa_etaria_75 a 79 anos', 
    'faixa_etaria_80 a 84 anos',
    'faixa_etaria_85 a 89 anos',
    'faixa_etaria_90 a 94 anos',
    'faixa_etaria_95 a 99 anos'
    ]

In [34]:
df_modelagem['prop_mais_60_anos'] = df_final[cols_mais_60].sum(axis=1)/df_final['total_zona']

In [35]:
cols_menos_25 = [
    'faixa_etaria_16 anos',
    'faixa_etaria_17 anos', 
    'faixa_etaria_18 anos', 
    'faixa_etaria_19 anos',
    'faixa_etaria_20 anos', 
    'faixa_etaria_21 a 24 anos'
    ]

In [36]:
df_modelagem['prop_menos_25_anos'] = df_final[cols_menos_25].sum(axis=1)/df_final['total_zona']

In [37]:
df_modelagem.head(10)

,nr_zona,prop_mulheres,prop_superior_completo,prop_sem_fundametal,prop_mais_60_anos,prop_menos_25_anos
0,1,0.521366,0.322225,0.154731,0.285246,0.071691
1,2,0.544536,0.432805,0.087887,0.324000,0.076005
2,3,0.501387,0.153666,0.229940,0.264833,0.102532
3,4,0.536943,0.257316,0.151490,0.302248,0.095024
4,5,0.546311,0.524865,0.057162,0.364926,0.059899
5,6,0.552555,0.429374,0.079574,0.318476,0.083337
6,20,0.543994,0.052214,0.301181,0.152717,0.160242
7,246,0.536280,0.362480,0.100450,0.300379,0.092375
8,247,0.534500,0.096444,0.263346,0.230581,0.119386
9,248,0.538055,0.098734,0.253334,0.197393,0.129274


In [38]:
gen_data_dir = solve_path('dados_modelagem', parent = 'data')

In [39]:
gen_data_dir

'/home/h-pgy/Desktop/Projetos/mulheres_bolsonaro/data/dados_modelagem'

In [40]:
df_modelo_fname = solve_path('var_exogenas_modelo.csv', gen_data_dir)

In [41]:
df_modelagem.to_csv(df_modelo_fname, sep=';', index=False,encoding='utf-8')